In [ ]:
pip install -U sentence-transformers

In [14]:
from datasets import load_dataset
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import average_precision_score
import time


In [15]:
# Load the WikiQA dataset
dataset = load_dataset("wiki_qa")
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

# Create DataFrames
train_df = pd.DataFrame({'question': train_data['question'], 'document': train_data['document_title'], 'answer': train_data['answer'], 'label': train_data['label']})
valid_df = pd.DataFrame({'question': valid_data['question'], 'document': valid_data['document_title'], 'answer': valid_data['answer'], 'label': valid_data['label']})
test_df = pd.DataFrame({'question': test_data['question'], 'document': test_data['document_title'], 'answer': test_data['answer'], 'label': test_data['label']})

# Display sample data
print(train_df.head())
print(valid_df.head())
print(test_df.head())

  0%|          | 0/3 [00:00<?, ?it/s]

                        question      document  \
0  how are glacier caves formed?  Glacier cave   
1  how are glacier caves formed?  Glacier cave   
2  how are glacier caves formed?  Glacier cave   
3  how are glacier caves formed?  Glacier cave   
4  how are glacier caves formed?  Glacier cave   

                                              answer  label  
0  A partly submerged glacier cave on Perito More...      0  
1          The ice facade is approximately 60 m high      0  
2          Ice formations in the Titlis glacier cave      0  
3  A glacier cave is a cave formed within the ice...      1  
4  Glacier caves are often called ice caves , but...      0  
                                      question          document  \
0  How are epithelial tissues joined together?  Tissue (biology)   
1  How are epithelial tissues joined together?  Tissue (biology)   
2  How are epithelial tissues joined together?  Tissue (biology)   
3  How are epithelial tissues joined together?  Tissue 

In [16]:
# Text preprocessing
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Apply text preprocessing to the DataFrames
train_df['question'] = train_df['question'].apply(preprocess_text)
train_df['document'] = train_df['document'].apply(preprocess_text)

valid_df['question'] = valid_df['question'].apply(preprocess_text)
valid_df['document'] = valid_df['document'].apply(preprocess_text)

test_df['question'] = test_df['question'].apply(preprocess_text)
test_df['document'] = test_df['document'].apply(preprocess_text)

# Display sample preprocessed data
print(train_df.head())
print(valid_df.head())
print(test_df.head())

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
               question      document  \
0  glacier caves formed  glacier cave   
1  glacier caves formed  glacier cave   
2  glacier caves formed  glacier cave   
3  glacier caves formed  glacier cave   
4  glacier caves formed  glacier cave   

                                              answer  label  
0  A partly submerged glacier cave on Perito More...      0  
1          The ice facade is approximately 60 m high      0  
2          Ice formations in the Titlis glacier cave      0  
3  A glacier cave is a cave formed within the ice...      1  
4  Glacier caves are often called ice caves , but...      0  
                             question        document  \
0  epithelial tissues joined together  tissue biology   
1  epithelial tissues joine

In [18]:
# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Tokenize and encode the questions and documents
embedding_dim = 768  # Update with the actual embedding dimension
train_questions_embeddings = model.encode(train_df['question'].tolist(), convert_to_tensor=True)
train_documents_embeddings = model.encode(train_df['document'].tolist(), convert_to_tensor=True)

valid_questions_embeddings = model.encode(valid_df['question'].tolist(), convert_to_tensor=True)
valid_documents_embeddings = model.encode(valid_df['document'].tolist(), convert_to_tensor=True)

test_questions_embeddings = model.encode(test_df['question'].tolist(), convert_to_tensor=True)
test_documents_embeddings = model.encode(test_df['document'].tolist(), convert_to_tensor=True)


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/637 [00:00<?, ?it/s]

Batches:   0%|          | 0/86 [00:00<?, ?it/s]

Batches:   0%|          | 0/86 [00:00<?, ?it/s]

Batches:   0%|          | 0/193 [00:00<?, ?it/s]

Batches:   0%|          | 0/193 [00:00<?, ?it/s]

In [33]:
import numpy as np
import torch
import time

# Assuming F is imported from torch.nn.functional
# If not, you need to import it: `import torch.nn.functional as F`

# Function to calculate cosine similarity
def calculate_cosine_similarity(query_embeddings, document_embeddings):
    query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
    document_embeddings = F.normalize(document_embeddings, p=2, dim=1)
    similarity_scores = torch.matmul(query_embeddings, document_embeddings.T)
    return similarity_scores.numpy()

# Function to calculate Mean Average Precision (MAP)
def calculate_map(y_true, y_scores):
    sorted_indices = np.argsort(y_scores)[::-1]
    num_correct = 0
    precision_at_k = 0
    y_true_array = np.array(y_true)

    for i, idx in enumerate(sorted_indices):
        num_correct += y_true_array[int(idx)].astype(int)
        precision_at_k += num_correct / (i + 1)

    if num_correct == 0:
        return 0

    return precision_at_k / num_correct

start_time = time.time()

# Generate random data for 100 queries (replace with actual embeddings)
num_queries = 100
embedding_dim = 256  
valid_query_embeddings = torch.randn((num_queries, embedding_dim))
valid_document_embeddings = torch.randn((len(valid_df), embedding_dim))
valid_labels = valid_df['label'].tolist()

# Calculate cosine similarities
cosine_similarities_matrix = calculate_cosine_similarity(valid_query_embeddings, valid_document_embeddings)

# Calculate MAP for each query
map_scores = []
for i in range(num_queries):
    query_scores = cosine_similarities_matrix[i]
    map_score = calculate_map(valid_labels, query_scores)
    map_scores.append(map_score)

    
end_time = time.time()
elapsed_time = end_time - start_time
    
# Calculate average MAP
average_map = np.mean(map_scores)

print(f"Mean Average Precision (MAP) on {num_queries} queries:" , average_map)
print(f"Elapsed time for evaluation: {elapsed_time} seconds")


Mean Average Precision (MAP) on 100 queries: 0.984857217066375
Elapsed time for evaluation: 0.7083127498626709 seconds
